In [1]:
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

y_data = np.array([
    [1, 0, 0],  # Unknown
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])


X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# INPUT DATA - HIDDEN LAYER - OUTPUT : 2 - 10 - 3([W: 2,10],[10,3])
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))
# BIAS
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)
model = tf.add(tf.matmul(L1, W2), b2)

# * tf.nn.softmax_cross... 함수는 one-hot vector로 사전 변환을 요구하나, 정수 실측값을 one-hot vector로 인코딩해주는
# tf.nn.sparse_softmax_cross.... 도 있다. -> one-hot vector의 길이가 길때(ex: 알파벳 등) 사용

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)


# train
init = tf.global_variables_initializer() #그래프 내 변수 초기화
sess = tf.Session()
sess.run(init)

epoch = 30
for step in range(epoch):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    tf.summary.histogram('Weights',W1)
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))


# predict
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('PREDICTION:', sess.run(prediction, feed_dict={X: x_data}))
print('REAL:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('ACCURACY: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


### 실험결과: epoch 수가 8 이하이면 매우 불안정하고, 10~20일때는 약 80%의 정확도를, 30이상 일때는 대체로 100%의 정확도를 보였다.
### 입력데이터가 지나치게 적어 EPOCH수와 정확도가 비례하지는 않았다

10 1.0244356
20 0.8391142
30 0.69961166
PREDICTION: [0 1 2 0 0 2]
REAL: [0 1 2 0 0 2]
ACCURACY: 100.00
